# Wikipedia Speedrun Project: Web Scraping!

In [2]:
# import relevant libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import random

#### Initialize important variables and functions


In [23]:
def getLinkData(link):
    Wikidata = requests.get(link)
    htmldata = BeautifulSoup(Wikidata.content, 'html.parser')
    result = htmldata.prettify()
    return result

def checkIfVisited(curlink, storelinks, startIdx, endIdx):
    for i in range(startIdx, endIdx):
        if("https://en.wikipedia.org" + curlink == storelinks[i]):
            return True
    return False



#### write function to recurse through hyperlinks!

In [24]:
storelinks = np.array(np.zeros(100000), dtype = str) # initialize memory
storelinks = storelinks.astype('U256') #convert to type that allows longer strings
startPage = "https://en.wikipedia.org/wiki/Casiopea"
curPageData = getLinkData(startPage) #call function that returns web page data
numLinks = 0
curIdx = 0
totNumSteps = 100 # number of iterations... control this for testing!
storeStepIndices = np.empty(shape = totNumSteps) 
storeSteps = []
storeStepNames = []
for numSteps in range(totNumSteps):
    links = [i for i in range(len(curPageData)) if curPageData.startswith("<a href=\"/wiki", i)] # isolate each hyperlink
    title = [i for i in range(len(curPageData)) if curPageData.startswith("<title", i)] #isolate title
    storeStepNames.append(curPageData[title[0] + 7 : title[0] + (curPageData[title[0] : title[0]+200]).find(' -')]) #add title to list
    for i in range(len(links)):
        curlink = curPageData[links[i] + 9 : links[i] + (curPageData[links[i]:links[i]+200]).find('" title')]
        if(curlink and curlink.find(':') < 0 and not checkIfVisited(curlink, storelinks, int(storeStepIndices[numSteps - 1]), numLinks)): #filter out: empty links, links already visited in the same iteration, and internal wikipedia links such as "Directory:" and "Help:"
            storelinks[numLinks] = "https://en.wikipedia.org" + curlink #store in proper hyperlink formatting
            storeSteps.append(numSteps) # replace with numSteps for actual output, something like 1 for testing unique hyperlinks
            numLinks += 1
    storeStepIndices[numSteps] = numLinks
    nextlink = storelinks[curIdx] if (getLinkData(storelinks[curIdx]) != curPageData) else storelinks[curIdx + 1]
    curPageData = getLinkData(nextlink)
    randIdx = 1 if (numSteps == 0) else (storeStepIndices[numSteps] - storeStepIndices[numSteps - 1])
    curIdx += random.randrange(0, int(randIdx))
storelinks = storelinks[storelinks != '0.0']
for i in range(len(storelinks)):                                #format and write to files
    storelinks[i] += (" " + str(storeSteps[i]) + '\n')
for i in range(len(storeStepNames)):
    storeStepNames[i] += (" " + "->")

file1 = open('output1.txt', 'w', encoding = 'utf-8')
file1.writelines(storelinks)
file1.close()

file2 = open('output2.txt', 'w', encoding = 'utf-8')
file2.writelines(storeStepNames)
file2.close()

print(len(storelinks))


29761


In [50]:
testlinkarr = []
testdata = requests.get("https://en.wikipedia.org/wiki/Casiopea")
testhtmldata = BeautifulSoup(testdata.content, 'html.parser')
for link in testhtmldata.find_all('a', href = True):
    curlink = link['href']
    if(curlink and curlink.find(':') < 0 and curlink.find('wiki') > 0):
        testlinkarr.append("https://en.wikipedia.org" + curlink)
    
print(testlinkarr)



['https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Casiopea', 'https://en.wikipedia.org/wiki/Casiopea', 'https://en.wikipedia.org/wiki/Casiopea', 'https://en.wikipedia.org/wiki/Cassiopeia_(disambiguation)', 'https://en.wikipedia.org//www.google.com/search?as_eq=wikipedia&q=%22Casiopea%22', 'https://en.wikipedia.org/wiki/Issei_Noro', 'https://en.wikipedia.org/wiki/Akira_Jimbo', 'https://en.wikipedia.org/wiki/Tokyo', 'https://en.wikipedia.org/wiki/Jazz_fusion', 'https://en.wikipedia.org/wiki/Jazz-funk', 'https://en.wikipedia.org/wiki/Smooth_jazz', 'https://en.wikipedia.org/wiki/City_pop', 'https://en.wikipedia.org/wiki/Alfa_Records', 'https://en.wikipedia.org/wiki/Polydor_Records', 'https://en.wikipedia.org/wiki/Pioneer_Entertainment', 'https://en.wikipedia.org/wiki/Pony_Canyon', 'https://en.wikipedia.org/wiki/Issei_Noro', 'https://en.wikipedia.org/wiki/Minoru_Mukaiya', 'https://en.wikipedia.org/wiki/Tetsuo_Sakurai', 'htt

current record:


1000 steps, 333585 unique links, 14m 2.7s runtime


100 steps, 33293 unique links, 1m 55.1s runtime


10 steps, 678 unique links, 43.8s runtime


1 step, 146 unique links, 31.2s runtime
